Create TFIDF matrix and save a reduced version for PCA

Simran Batra

DS 5559

# Configuration

In [1]:
db_name = 'pitchfork.db'
db_name2 = "pitchfork_adjectives"

OHCO = ['Title', 'Artist', 'Score', 'Date','Genre', 'Author', 'para_num', 'sent_num', 'token_num']
TITLE = OHCO[:1]
ARTIST = OHCO[:2]
SCORE = OHCO[:3]
DATE = OHCO[:4]
GENRE = OHCO[:5]
AUTHOR = OHCO[:6]
PARAS = OHCO[:7]
SENTS = OHCO[:8]

BAG = AUTHOR

## Libraries

In [2]:
import sqlite3
import pandas as pd
import numpy as np

# Create DTM

## Load data

In [3]:
with sqlite3.connect(db_name) as db:
    tokens = pd.read_sql('SELECT * FROM token', db, index_col=OHCO)
    vocab = pd.read_sql('SELECT * FROM vocab', db, index_col='term_id')
    docs =  pd.read_sql('SELECT * FROM doc', db, index_col=AUTHOR)

## Create word mask

In [5]:
WORDS = (tokens.punc == 0) & (tokens.num == 0)

## Extract BOW from tokens

In [6]:
BOW = tokens[WORDS].groupby(BAG + ['term_id'])['term_id'].count()

## Convert BOW to DTM

In [7]:
DTM = BOW.unstack().fillna(0)

## Create Bags table

In [8]:
bags = pd.DataFrame(index = DTM.index)

In [9]:
DTM = DTM.reset_index(drop=True)
DTM.index.name = 'bag_id'

# Compute Term Frequencies and Weights

## Compute IDF

In [10]:
N_docs = DTM.shape[0]
vocab['df'] = DTM[DTM > 0].count()
vocab['idf'] = np.log10(N_docs / vocab.df)

In [11]:
vocab[vocab.stop==0].sort_values('n', ascending=False).head(500)\
    .sort_values('idf', ascending=False).head(20)

,term_str,n,p,port_stem,stop,df,idf
term_id,,,,,,,
71581,raps,858,0.000175,rap,0,505.0,1.141310
88818,techno,969,0.000197,techno,0,535.0,1.116248
23539,disco,924,0.000188,disco,0,593.0,1.071547
24254,dj,956,0.000195,dj,0,604.0,1.063564
86782,sun,917,0.000187,sun,0,614.0,1.056433
9516,blues,1024,0.000208,blue,0,616.0,1.055021
49029,la,1040,0.000212,la,0,626.0,1.048027
88267,talk,913,0.000186,talk,0,648.0,1.033026
71533,rapper,1132,0.000230,rapper,0,653.0,1.029688


## Compute TFIDF

In [12]:
TFIDF = DTM * vocab['idf']

In [13]:
vocab[TFIDF.sum() == 0] # test for stop words

,term_str,n,p,port_stem,stop,df,idf
term_id,,,,,,,
0,a,142517,2.900697e-02,a,1,6992.0,0.0
541,acheive,1,2.035334e-07,acheiv,0,NaN,NaN
2995,and,138244,2.813727e-02,and,1,6992.0,0.0
24851,dossier,1,2.035334e-07,dossier,0,NaN,NaN
61931,of,147904,3.010340e-02,of,1,6992.0,0.0
89564,the,255840,5.207198e-02,the,1,6992.0,0.0
100272,wulfstan,1,2.035334e-07,wulfstan,0,NaN,NaN
100597,yatton,1,2.035334e-07,yatton,0,NaN,NaN


## Add stats to Vocab

In [14]:
vocab['tfidf_sum'] = TFIDF.sum()
vocab['tfidf_mean'] = TFIDF.mean()
vocab['tfidf_max'] = TFIDF.max()

# Get Top words and Trim Matrix

In [15]:
def get_top_terms(vocab, no_stops=True, sort_col='n', k=1500):
    if no_stops:
        V = vocab[vocab.stop == 0]
    else:
        V = vocab
    return V.sort_values(sort_col, ascending=False).head(k)

## Filter proper nouns

In [16]:
proper_nouns = tokens.loc[tokens.pos == 'NNP', 'term_id'].unique()

In [17]:
top_n = 1500
# TOPV = get_top_terms(vocab, sort_col='n')
TOPV = get_top_terms(vocab.loc[~vocab.index.isin(proper_nouns)], sort_col='n')
TOPV

,term_str,n,p,port_stem,stop,df,idf,tfidf_sum,tfidf_mean,tfidf_max
term_id,,,,,,,,,,
73338,released,2306,0.000469,releas,0,1764.0,0.598103,1379.225146,0.197258,6.579131
78500,seem,2125,0.000433,seem,0,1715.0,0.610337,1296.966752,0.185493,3.662024
72221,recent,1552,0.000316,recent,0,1323.0,0.723042,1122.160525,0.160492,3.615208
50258,length,1387,0.000282,length,0,1233.0,0.753638,1045.296382,0.149499,3.014553
95436,used,1187,0.000242,use,0,1029.0,0.832186,987.804835,0.141276,3.328744
29230,exactly,1128,0.000230,exactli,0,987.0,0.850284,959.120653,0.137174,3.401137
82923,songwriting,1061,0.000216,songwrit,0,906.0,0.887473,941.609088,0.134669,4.437366
78502,seemed,1009,0.000205,seem,0,835.0,0.922915,931.221179,0.133184,5.537490
28166,entirely,884,0.000180,entir,0,789.0,0.947524,837.611584,0.119796,2.842573


## Create tf-idf matrix

In [18]:
tfidf_small = TFIDF[TOPV.index].stack().to_frame().rename(columns={0:'w'})

# Save to SQL

In [19]:
with sqlite3.connect(db_name) as db:
    vocab.to_sql('vocab', db, if_exists='replace', index=True)
    tokens.to_sql('token', db, if_exists='replace', index=True)
    docs.to_sql('doc', db, if_exists='replace', index=True)
    tfidf_small.to_sql('tfidf_small', db, if_exists='replace', index=True)
    bags.reset_index().to_sql('bag', db, if_exists='replace', index=True, index_label='bag_id')

## Filter only adjectives

In [20]:
adjectives = tokens.loc[tokens.pos == 'JJ', 'term_id'].unique()
top_n = 1500
TOPV = get_top_terms(vocab.loc[vocab.index.isin(adjectives)], sort_col='n')
tfidf_small = TFIDF[TOPV.index].stack().to_frame().rename(columns={0:'w'})
TOPV

,term_str,n,p,port_stem,stop,df,idf,tfidf_sum,tfidf_mean,tfidf_max
term_id,,,,,,,,,,
50821,like,28205,0.005741,like,0,6676.0,0.020085,566.439754,0.081013,0.441872
1863,album,16433,0.003345,album,0,5585.0,0.097578,1603.308097,0.229306,1.853987
62380,one,14586,0.002969,one,0,5641.0,0.093245,1359.889745,0.194492,2.237888
58807,music,14489,0.002949,music,0,5180.0,0.130272,1887.375808,0.269934,2.996248
82802,songs,12412,0.002526,song,0,4914.0,0.153166,1901.099712,0.271896,3.063325
29039,even,9626,0.001959,even,0,4878.0,0.156360,1504.961378,0.215240,2.189035
82752,song,9569,0.001948,song,0,4385.0,0.202632,1938.983904,0.277315,4.863164
83166,sound,9425,0.001918,sound,0,4754.0,0.167542,1578.918091,0.225818,1.842965
60149,new,9055,0.001843,new,0,4417.0,0.199474,1805.838306,0.258272,3.790006


Looks bad!  I checked the adjectives to see if they were filtering properly.

In [21]:
adjectives = tokens.loc[tokens.pos == 'JJ', 'term_str'].unique()
adjectives_df = pd.DataFrame(adjectives)
adjectives_df

,0
0,enviable
1,able
2,familiar
3,other
4,little
5,adrift
6,contemporary
7,stylistic
8,number
9,scrappy


Looks like adjectives, so I compared a few of the top values that didn't seem like adjectives to see if they were actually coded as adjectives.

In [22]:
w = ["like", "album", "music", "one", "songs"]
for word in w:
    print(adjectives_df[adjectives_df[0] == word])

         0
1114  like
          0
1122  album
         0
286  music
       0
160  one
         0
499  songs


The words are actually coded as adjectives at least once.  Looks like this is just a case of bad pos tagging.

## Save adjectives to db

In [23]:
with sqlite3.connect(db_name2) as db:
    vocab.to_sql('vocab', db, if_exists='replace', index=True)
    tokens.to_sql('token', db, if_exists='replace', index=True)
    docs.to_sql('doc', db, if_exists='replace', index=True)
    tfidf_small.to_sql('tfidf_small', db, if_exists='replace', index=True)
    bags.reset_index().to_sql('bag', db, if_exists='replace', index=True, index_label='bag_id')